In [1]:
# Import all the needed libraries
import pyterrier as pt
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import algos
import os
if not pt.started():
    pt.init()

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [2]:
# Load the dataset
dataset = pt.datasets.get_dataset("trec-deep-learning-passages")

In [3]:
index_ref = pt.IndexRef.of(os.getcwd() + "./passage_index/data.properties")

In [4]:
index = pt.IndexFactory.of(index_ref)

20:20:35.814 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1,9 GiB of memory would be required.


In [5]:
pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=["WMODEL:Tf", "WMODEL:PL2"])

In [6]:
train_topics = dataset.get_topics("train").iloc[0:100]
train_qrels = dataset.get_qrels("train")

20:20:35.894 [main] WARN org.terrier.applications.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (windows-1252). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8


In [7]:
import time
start_time = time.time()

In [8]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=400)
rf_pipe = pipeline >> pt.ltr.apply_learned_model(rf)
rf_pipe.fit(train_topics, train_qrels)

In [9]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 3199.8517322540283 seconds ---


In [10]:
pt.Experiment([rf_pipe], dataset.get_topics("test-2019"), 
  dataset.get_qrels("test-2019"), ["map"], names=["LTR"])

21:14:03.743 [main] WARN org.terrier.applications.batchquerying.TRECQuery - trec.encoding is not set; resorting to platform default (windows-1252). Retrieval may be platform dependent. Recommend trec.encoding=UTF-8


,name,map
0,LTR,0.086134


In [ ]:
# Start 20:20

In [24]:
# Here comes a visualization of the graph
# https://stackoverflow.com/questions/40155128/plot-trees-for-a-random-forest-in-python-with-scikit-learn